# Elektron WebSocket API MRN Example with Python

## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

MRN aims for replacing the legacy News 2K (N2_UBMS and N2_STORY).

### MRN Data behavior
MRN is published over Elektron using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (ESDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/mrn_process.png"/>

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [2]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib

In [3]:
# TREP connection variables

hostname = '172.20.33.30'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [4]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_news_messages = []

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

 

In [5]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_TRNA'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [6]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [7]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [8]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [13]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(90)
    web_socket_app.close()

Connecting to WebSocket ws://172.20.33.30:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"10.42.84.234"
    },
    "Name":"root"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":61430,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"10.42.84.234",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":1,
        "SupportPauseResume":1,
        "SupportStandby":1,
        "SupportViewRequests":1
      },
      "Name":"root"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Open",
      "Text":"Login accepted by h

In [14]:
_news_messages

[{'analytics': {'analyticsScores': [{'assetClass': 'CMPNY',
     'assetCodes': ['P:4295877060', 'R:9613.T'],
     'assetId': '4295877060',
     'assetName': 'NTT Data Corp',
     'brokerAction': 'UNDEFINED',
     'firstMentionSentence': 18,
     'linkedIds': [{'idPosition': 0,
       'linkedId': 'tr:L4N2AH0MW_2002172t7Lt/UgBhCNRRn74Xz2OT5nqFECmV9LWr+qBp'}],
     'noveltyCounts': [{'itemCount': 1, 'window': '12H'},
      {'itemCount': 1, 'window': '24H'},
      {'itemCount': 1, 'window': '3D'},
      {'itemCount': 1, 'window': '5D'},
      {'itemCount': 1, 'window': '7D'}],
     'priceTargetIndicator': 'UNDEFINED',
     'relevance': 0.5,
     'sentimentClass': 0,
     'sentimentNegative': 0.016526,
     'sentimentNeutral': 0.924055,
     'sentimentPositive': 0.0594192,
     'sentimentWordCount': 81,
     'volumeCounts': [{'itemCount': 1, 'window': '12H'},
      {'itemCount': 1, 'window': '24H'},
      {'itemCount': 1, 'window': '3D'},
      {'itemCount': 1, 'window': '5D'},
      {'item

In [11]:
print("first 10 analytics\n")
for analytic in _news_messages[:10]:
    if analytic["analytics"]:
        print(analytic["analytics"])

first 10 analytics



In [34]:
test_analytics = _news_messages[2:3]
test_analytics

[{'analytics': {'analyticsScores': [{'assetClass': 'CMPNY',
     'assetCodes': ['P:4295886989', 'R:MSRS.MM'],
     'assetId': '4295886989',
     'assetName': 'MOESK PAO',
     'brokerAction': 'UNDEFINED',
     'firstMentionSentence': 1,
     'linkedIds': [],
     'noveltyCounts': [{'itemCount': 0, 'window': '12H'},
      {'itemCount': 0, 'window': '24H'},
      {'itemCount': 0, 'window': '3D'},
      {'itemCount': 0, 'window': '5D'},
      {'itemCount': 0, 'window': '7D'}],
     'priceTargetIndicator': 'UNDEFINED',
     'relevance': 1.0,
     'sentimentClass': 0,
     'sentimentNegative': 0.35394,
     'sentimentNeutral': 0.366123,
     'sentimentPositive': 0.279937,
     'sentimentWordCount': 63,
     'volumeCounts': [{'itemCount': 0, 'window': '12H'},
      {'itemCount': 0, 'window': '24H'},
      {'itemCount': 0, 'window': '3D'},
      {'itemCount': 0, 'window': '5D'},
      {'itemCount': 0, 'window': '7D'}]}],
   'newsItem': {'bodySize': 306,
    'companyCount': 1,
    'exchangeAct

In [35]:
analytic_scores_group = test_analytics[0]["analytics"]["analyticsScores"]
analytic_scores_group

[{'assetClass': 'CMPNY',
  'assetCodes': ['P:4295886989', 'R:MSRS.MM'],
  'assetId': '4295886989',
  'assetName': 'MOESK PAO',
  'brokerAction': 'UNDEFINED',
  'firstMentionSentence': 1,
  'linkedIds': [],
  'noveltyCounts': [{'itemCount': 0, 'window': '12H'},
   {'itemCount': 0, 'window': '24H'},
   {'itemCount': 0, 'window': '3D'},
   {'itemCount': 0, 'window': '5D'},
   {'itemCount': 0, 'window': '7D'}],
  'priceTargetIndicator': 'UNDEFINED',
  'relevance': 1.0,
  'sentimentClass': 0,
  'sentimentNegative': 0.35394,
  'sentimentNeutral': 0.366123,
  'sentimentPositive': 0.279937,
  'sentimentWordCount': 63,
  'volumeCounts': [{'itemCount': 0, 'window': '12H'},
   {'itemCount': 0, 'window': '24H'},
   {'itemCount': 0, 'window': '3D'},
   {'itemCount': 0, 'window': '5D'},
   {'itemCount': 0, 'window': '7D'}]}]

In [36]:
for analytic_score in analytic_scores_group:
    if analytic_score["assetClass"]:
        print("assetClass: ", analytic_score["assetClass"])
        print("assetCodes: ", analytic_score["assetCodes"])
        print("Broker Action: ", analytic_score["brokerAction"])
        print("Rel: ",analytic_score["relevance"])
        print("Sent: ", analytic_score["sentimentClass"])
        print("Pos: ", analytic_score["sentimentPositive"])
        print("Neut: ", analytic_score["sentimentNeutral"])
        print("Neg: ", analytic_score["sentimentNegative"])
        print("Price Target: ", analytic_score["priceTargetIndicator"])
        print("1st Mentn: ", analytic_score["firstMentionSentence"])
        print("#Sent wds/Tkns: ",analytic_score["sentimentWordCount"])
        print("Volume Count: ", analytic_score["volumeCounts"])
        print("--------------------------------------------------------")

assetClass:  CMPNY
assetCodes:  ['P:4295886989', 'R:MSRS.MM']
Broker Action:  UNDEFINED
Rel:  1.0
Sent:  0
Pos:  0.279937
Neut:  0.366123
Neg:  0.35394
Price Target:  UNDEFINED
1st Mentn:  1
#Sent wds/Tkns:  63
Volume Count:  [{'itemCount': 0, 'window': '12H'}, {'itemCount': 0, 'window': '24H'}, {'itemCount': 0, 'window': '3D'}, {'itemCount': 0, 'window': '5D'}, {'itemCount': 0, 'window': '7D'}]
--------------------------------------------------------


In [37]:
news_item_groups = test_analytics[0]["analytics"]["newsItem"]
news_item_groups

{'bodySize': 306,
 'companyCount': 1,
 'exchangeAction': 'UNDEFINED',
 'headlineTag': 'BRIEF',
 'marketCommentary': False,
 'sentenceCount': 5,
 'wordCount': 80}

In [38]:
print("Exchange Action: ", news_item_groups["exchangeAction"])

Exchange Action:  UNDEFINED


In [39]:
news_item  = test_analytics[0]["newsItem"]
news_item

{'dataType': 'News',
 'feedFamilyCode': 'tr',
 'headline': "BRIEF-Russia's MOESK To Conduct Book Building For RUB 10 Bln Bonds on Feb 18 - Source",
 'language': 'en',
 'metadata': {'altId': 'nL8N2AE359',
  'audiences': ['NP:E', 'NP:PSC', 'NP:RNP', 'NP:PCO'],
  'feedTimestamp': '2020-02-14T09:56:17.743Z',
  'firstCreated': '2020-02-14T09:56:17.000Z',
  'isArchive': False,
  'takeSequence': 1},
 'provider': 'NS:RTRS',
 'sourceId': 'L8N2AE359_2002142oGeduoYErJ0f3PCqmI6oC1trOcRpI8uqRL6vk',
 'sourceTimestamp': '2020-02-14T09:56:17.000Z',
 'subjects': ['M:1NZ',
  'M:Z',
  'A:T',
  'B:1824',
  'B:183',
  'G:38',
  'A:2',
  'A:U',
  'E:1',
  'B:181',
  'B:182',
  'G:1',
  'G:A',
  'G:AJ',
  'G:B',
  'G:E',
  'G:K',
  'G:S',
  'R:MSRS.MM',
  'P:4295886989',
  'MCCL:OEC',
  'MCC:OEC',
  'M:1QD',
  'N2:GDY1',
  'N2:CMPNY',
  'N2:CORPD',
  'N2:UTLFF',
  'N2:ELUT',
  'N2:RU',
  'N2:DBT',
  'N2:CDM',
  'N2:BACT',
  'N2:UTIL',
  'N2:ELEU',
  'N2:EMRG',
  'N2:EUROP',
  'N2:CISC',
  'N2:CEEU',
  'N2:NA

In [40]:
print("Headline: ", news_item["headline"])
print("Regional Timestamp: ", news_item["sourceTimestamp"])
print("Attrib: ", news_item["provider"])
print("Item Type: ", news_item["urgency"], " : ", (lambda item_type: "alert" if 1 else "article")(news_item["urgency"]))

Headline:  BRIEF-Russia's MOESK To Conduct Book Building For RUB 10 Bln Bonds on Feb 18 - Source
Regional Timestamp:  2020-02-14T09:56:17.000Z
Attrib:  NS:RTRS
Item Type:  3  :  alert


## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.thomsonreuters.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with Elektron WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).